Dependencies

In [ ]:
!pip install torch torchaudio tqdm soundfile


Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## ✅ Configuration persistante (Google Drive)
Tout est stocké dans Drive pour survivre aux resets Colab : dataset, checkpoints, exports ONNX, logs.

In [ ]:
# === Dossier persistant dans Google Drive ===
import os, time, shutil, json, pathlib

PROJECT_DIR = "/content/drive/MyDrive/ProjetIA_Voice"
DATASET_DIR = os.path.join(PROJECT_DIR, "dataset")          # dossier dataset (persistant)
CHECKPOINT_DIR = os.path.join(PROJECT_DIR, "checkpoints")   # poids/optimiseur
EXPORT_DIR = os.path.join(PROJECT_DIR, "exports")           # ONNX / TensorRT
LOG_DIR = os.path.join(PROJECT_DIR, "logs")

for d in [PROJECT_DIR, DATASET_DIR, CHECKPOINT_DIR, EXPORT_DIR, LOG_DIR]:
    os.makedirs(d, exist_ok=True)

# ✅ Chemins utilisés partout (à ne plus mettre dans /content)
DATA_DIR = os.path.join(DATASET_DIR, "voice_dataset_final")   # <- ton dataset doit être ici (classes/xxx.wav)
MODEL_SAVE_PATH = os.path.join(CHECKPOINT_DIR, "voice_model.pth")
OPTIMIZER_SAVE_PATH = os.path.join(CHECKPOINT_DIR, "voice_optimizer.pth")
ONNX_PATH = os.path.join(EXPORT_DIR, "voice_model.onnx")
ONNX_SINGLE_PATH = os.path.join(EXPORT_DIR, "voice_model_single.onnx")

print("✅ PROJECT_DIR:", PROJECT_DIR)
print("✅ DATA_DIR:", DATA_DIR)
print("✅ MODEL_SAVE_PATH:", MODEL_SAVE_PATH)
print("✅ EXPORT_DIR:", EXPORT_DIR)


✅ PROJECT_DIR: /content/drive/MyDrive/ProjetIA_Voice
✅ DATA_DIR: /content/drive/MyDrive/ProjetIA_Voice/dataset/voice_dataset_final
✅ MODEL_SAVE_PATH: /content/drive/MyDrive/ProjetIA_Voice/checkpoints/voice_model.pth
✅ EXPORT_DIR: /content/drive/MyDrive/ProjetIA_Voice/exports


## 📦 Dataset : upload / extraction (optionnel)
Si ton dataset est un ZIP, uploade-le puis décompresse-le directement dans Drive (persistant).

In [ ]:
# (Optionnel) Upload d'un ZIP dataset puis extraction dans Drive
# Attendu après extraction : DATA_DIR = .../dataset/voice_dataset_final/ (avec sous-dossiers = classes)
from google.colab import files
import zipfile, os, shutil

def upload_and_extract_dataset_zip():
    uploaded = files.upload()  # sélectionne voice_dataset_final.zip par exemple
    for fn in uploaded.keys():
        if fn.lower().endswith(".zip"):
            zip_path = os.path.join(DATASET_DIR, fn)
            shutil.move(fn, zip_path)
            print("✅ ZIP copié dans Drive:", zip_path)

            # extraction
            with zipfile.ZipFile(zip_path, 'r') as z:
                z.extractall(DATASET_DIR)
            print("✅ ZIP extrait dans:", DATASET_DIR)
        else:
            print("ℹ️ Fichier ignoré (pas un .zip):", fn)

# Décommente pour lancer :
upload_and_extract_dataset_zip()

# Vérification rapide
if os.path.isdir(DATA_DIR):
    print("✅ Dataset trouvé. Classes :", sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR,d))])[:20])
else:
    print("⚠️ DATA_DIR introuvable :", DATA_DIR)
    print("➡️ Place ton dataset ici (dans Drive) ou utilise upload_and_extract_dataset_zip()")


Saving voice_dataset_final.zip to voice_dataset_final.zip
✅ ZIP copié dans Drive: /content/drive/MyDrive/ProjetIA_Voice/dataset/voice_dataset_final.zip
✅ ZIP extrait dans: /content/drive/MyDrive/ProjetIA_Voice/dataset
✅ Dataset trouvé. Classes : ['avance', 'droite', 'gauche', 'recule', 'stop']


Code

In [ ]:
import os
print("Contenu du dossier /content :")
print(os.listdir("/content"))


Contenu du dossier /content :
['.config', 'drive', 'sample_data']


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from tqdm import tqdm

# ========================
# ⚙️ Paramètres
# ========================
# DATA_DIR est défini dans la cellule Drive (persistant)
# DATA_DIR = "..."

BATCH_SIZE = 16
EPOCHS = 500
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ========================
# 📦 Dataset
# ========================
class VoiceCommandDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir):
        self.data, self.labels = [], []
        self.classes = sorted(os.listdir(data_dir))

        self.mel = MelSpectrogram(sample_rate=16000, n_mels=64)
        self.db = AmplitudeToDB()

        for label, cls in enumerate(self.classes):
            cls_path = os.path.join(data_dir, cls)
            for file in os.listdir(cls_path):
                if file.endswith(".wav"):
                    self.data.append(os.path.join(cls_path, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path = self.data[idx]

        audio, sr = sf.read(path, dtype="float32")  # audio: (N,) ou (N, C)
        if audio.ndim == 2:
            audio = audio.mean(axis=1)  # mono

        # resampling si besoin (idéalement ton dataset est déjà en 16k)
        if sr != 16000:
            # Option simple: on ignore ici; mieux = resampler torchaudio si tu veux
            pass

        waveform = torch.from_numpy(audio).unsqueeze(0)  # (1, N)

        # Normalisation durée 1s
        target_len = 16000
        if waveform.shape[1] < target_len:
            waveform = F.pad(waveform, (0, target_len - waveform.shape[1]))
        else:
            waveform = waveform[:, :target_len]

        mel_spec = self.db(self.mel(waveform))  # (1, 64, T)
        return mel_spec, torch.tensor(self.labels[idx], dtype=torch.long)


# ========================
# 🧠 Modèle CNN
# ========================
class VoiceNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )

        # Dynamically calculate the flattened size for the linear layer
        # Use a dummy input with an assumed max time_steps (e.g., 80)
        # that could be produced by collate_fn's padding.
        # This ensures the linear layer's input features match the conv output.
        with torch.no_grad():
            dummy_input_shape = (1, 1, 64, 80)  # (batch_size, channels, n_mels, time_steps)
            dummy_input = torch.zeros(dummy_input_shape)
            dummy_output = self.conv(dummy_input)
            flattened_size = dummy_output.view(dummy_output.size(0), -1).shape[1]

        self.fc = nn.Sequential(
            nn.Linear(flattened_size, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

Padding auto pour avoir les memes longueurs de spectogrammes

In [ ]:
import torch.nn.functional as F

def collate_fn(batch):
    specs, labels = zip(*batch)
    # Trouver la taille max dans le batch
    max_len = max([s.shape[-1] for s in specs])
    padded_specs = []
    for s in specs:
        pad_size = max_len - s.shape[-1]
        s = F.pad(s, (0, pad_size))  # padding sur la dimension temporelle
        padded_specs.append(s)
    specs = torch.stack(padded_specs)
    labels = torch.tensor(labels)
    return specs, labels


In [ ]:
!pip -q install soundfile
import soundfile as sf


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

# Dépendances audio stables (PAS de torchcodec)
!apt-get update -qq
!apt-get install -qq ffmpeg
!pip -q install soundfile

# ========================
# ✨ Entraînement
# ========================
dataset = VoiceCommandDataset(DATA_DIR)
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

model = VoiceNet(num_classes=len(dataset.classes)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ========================
# 🔁 Reprise d'entraînement (resume)
# ========================
RESUME_TRAINING = True
RESUME_FROM_EPOCH = 20  # si tu as fini l'époque 20, mets 20 pour repartir à 21

start_epoch = 0

if RESUME_TRAINING and os.path.exists(MODEL_SAVE_PATH):
    print(f"📦 Chargement du modèle depuis : {MODEL_SAVE_PATH}")
    model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))

    if os.path.exists(OPTIMIZER_SAVE_PATH):
        print(f"📦 Chargement de l'optimiseur depuis : {OPTIMIZER_SAVE_PATH}")
        optimizer.load_state_dict(torch.load(OPTIMIZER_SAVE_PATH, map_location=DEVICE))

    start_epoch = RESUME_FROM_EPOCH
    print(f"✅ Reprise de l'entraînement à partir de l'époque {start_epoch + 1}")

elif RESUME_TRAINING and not os.path.exists(MODEL_SAVE_PATH):
    print(f"⚠️ Aucun modèle trouvé à : {MODEL_SAVE_PATH}")
    print("➡️ Démarrage d'un nouvel entraînement.")
    start_epoch = 0
else:
    print("➡️ Démarrage d'un nouvel entraînement.")
    start_epoch = 0

print(f"🧠 Entraînement sur {DEVICE} ({len(dataset)} échantillons) ...")

# ========================
# 🚀 Boucle d'entraînement
# ========================
for epoch in range(start_epoch, EPOCHS):
    model.train()
    total_loss = 0.0

    for X, y in tqdm(loader, desc=f"Époque {epoch+1}/{EPOCHS}"):
        X, y = X.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / max(1, len(loader))
    print(f"💡 Époque {epoch+1}: loss = {avg_loss:.4f}")

    # Sauvegardes (idéalement dans Drive)
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    torch.save(optimizer.state_dict(), OPTIMIZER_SAVE_PATH)
    print(f"✅ Modèle + optimiseur sauvegardés après l'époque {epoch+1}")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
⚠️ Aucun modèle trouvé à : /content/drive/MyDrive/ProjetIA_Voice/checkpoints/voice_model.pth
➡️ Démarrage d'un nouvel entraînement.
🧠 Entraînement sur cuda (1450 échantillons) ...


Époque 1/500: 100%|██████████| 91/91 [00:08<00:00, 11.24it/s]


💡 Époque 1: loss = 1.0986
✅ Modèle + optimiseur sauvegardés après l'époque 1


Époque 2/500: 100%|██████████| 91/91 [00:05<00:00, 15.18it/s]


💡 Époque 2: loss = 0.0984
✅ Modèle + optimiseur sauvegardés après l'époque 2


...

💡 Époque 498: loss = 0.0468
✅ Modèle + optimiseur sauvegardés après l'époque 498


Époque 499/500: 100%|██████████| 91/91 [00:06<00:00, 14.23it/s]


💡 Époque 499: loss = 0.0306
✅ Modèle + optimiseur sauvegardés après l'époque 499


Époque 500/500: 100%|██████████| 91/91 [00:05<00:00, 15.22it/s]


💡 Époque 500: loss = 0.0010
✅ Modèle + optimiseur sauvegardés après l'époque 500


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X, y in loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        outputs = model(X)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

print("\n✅ Évaluation du modèle :")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))



✅ Évaluation du modèle :
              precision    recall  f1-score   support

      avance       1.00      1.00      1.00       290
      droite       1.00      1.00      1.00       290
      gauche       1.00      1.00      1.00       290
      recule       1.00      1.00      1.00       290
        stop       1.00      1.00      1.00       290

    accuracy                           1.00      1450
   macro avg       1.00      1.00      1.00      1450
weighted avg       1.00      1.00      1.00      1450



Passage du modele sur cpu et en mode eval


In [ ]:
import torch

model.cpu()
model.eval()

VoiceNet(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=10240, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=5, bias=True)
  )
)

Vérification de la taille réelle de l’entrée


In [ ]:
x, _ = dataset[0]
print("Shape MelSpectrogram :", x.shape)


Shape MelSpectrogram : torch.Size([1, 64, 81])


(OPTIONNEL) Quantization PyTorch (INT8 CPU)

In [ ]:
import torch.quantization as tq

torch.backends.quantized.engine = "fbgemm"
model.qconfig = tq.get_default_qconfig("fbgemm")

model_prepared = tq.prepare(model, inplace=False)

# Calibration rapide (sans DataLoader)
with torch.no_grad():
    for _ in range(20):
        model_prepared(torch.randn(1, 1, 64, x.shape[-1]))

model_quantized = tq.convert(model_prepared, inplace=False)

print("✅ Modèle quantifié INT8 (CPU)")


✅ Modèle quantifié INT8 (CPU)


/tmp/ipython-input-2162412526.py:6: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_prepared = tq.prepare(model, inplace=False)
/usr/local/lib/python3.12/dist-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyT

EXPORT ONNX

In [ ]:
!pip install onnx onnxscript


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 18.0 MB/s eta 0:00:00


In [ ]:
!pip -q install onnx onnxscript
dummy_input = torch.randn(1, 1, 64, x.shape[-1])

torch.onnx.export(
    model,  # FP32
    dummy_input,
    ONNX_PATH,
    input_names=["input"],
    output_names=["logits"],
    dynamic_axes={
        "input": {0: "batch"},
        "logits": {0: "batch"}
    }
)

print("✅ Export ONNX terminé : voice_model.onnx")

/tmp/ipython-input-995897952.py:4: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0125 22:47:32.658000 506 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `VoiceNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `VoiceNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/BaseConverter.h:68: adapter_lookup: Assertion `false`

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 1 of general pattern rewrite rules.
✅ Export ONNX terminé : voice_model.onnx


TESTER LE ONNX

In [ ]:
!pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.2 MB/s eta 0:00:00


In [ ]:
import os, glob

# Cherche tous les .onnx dans /content et Drive
paths = glob.glob("/content/**/*.onnx", recursive=True) + glob.glob("/content/drive/MyDrive/**/*.onnx", recursive=True)

print("🔎 ONNX trouvés :")
for p in paths[:50]:
    print(" -", p)

print("\nTotal:", len(paths))


🔎 ONNX trouvés :
 - /content/drive/MyDrive/ProjetIA_Voice/voice_model.onnx
 - /content/drive/MyDrive/ProjetIA_Voice/exports/voice_model.onnx
 - /content/drive/MyDrive/ProjetIA_Voice/voice_model.onnx
 - /content/drive/MyDrive/ProjetIA_Voice/exports/voice_model.onnx

Total: 4


In [ ]:
import onnxruntime as ort
import numpy as np

ONNX_PATH = "/content/drive/MyDrive/ProjetIA_Voice/exports/voice_model.onnx"

sess = ort.InferenceSession(ONNX_PATH)

# ⚠️ T doit correspondre à ton export
T = 81  # si tu n'es pas sûr, je te montre juste après comment le récupérer
x = np.random.randn(1, 1, 64, T).astype(np.float32)

y = sess.run(None, {"input": x})[0]
print("✅ Output shape:", y.shape)
print("✅ Logits:", y)


✅ Output shape: (1, 5)
✅ Logits: [[-3.767791  -6.7623363  1.768387   0.562924  -3.7057798]]


In [ ]:
import onnxruntime as ort
import numpy as np

sess = ort.InferenceSession(ONNX_SINGLE_PATH)
y = sess.run(
    None,
    {"input": dummy_input.numpy()}
)

print("ONNX output shape :", y[0].shape)

NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from /content/drive/MyDrive/ProjetIA_Voice/exports/voice_model_single.onnx failed:Load model /content/drive/MyDrive/ProjetIA_Voice/exports/voice_model_single.onnx failed. File doesn't exist

Téléchargement de files

In [ ]:

from google.colab import files
# Téléchargements depuis Drive
files.download(MODEL_SAVE_PATH)
files.download(ONNX_SINGLE_PATH if os.path.exists(ONNX_SINGLE_PATH) else ONNX_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ✅ Conversion ONNX en "single file" (sans .onnx.data), compatible toutes versions
!pip -q install onnx

import onnx
import os

m = onnx.load(ONNX_PATH)
onnx.save_model(m, ONNX_SINGLE_PATH, save_as_external_data=False)

print("✅ ONNX single file :", ONNX_SINGLE_PATH)
print("📁 Fichiers export :", [f for f in os.listdir(EXPORT_DIR) if f.startswith("voice_model")])


✅ ONNX single file : /content/drive/MyDrive/ProjetIA_Voice/exports/voice_model_single.onnx
📁 Fichiers export : ['voice_model.onnx.data', 'voice_model.onnx', 'voice_model_single.onnx']


## 🚀 TensorRT sur Jetson (à faire quand tu auras la carte)
Tu peux convertir l'ONNX en engine TensorRT directement sur la Jetson. Sur Jetson Nano, privilégie FP16. INT8 nécessite une calibration TensorRT.

In [ ]:
# Commandes à exécuter SUR LA JETSON (terminal), pas dans Colab.

# 1) Vérifier la version TensorRT / trtexec
# which trtexec
# trtexec --version

# 2) FP16 (souvent le plus simple)
# trtexec --onnx=voice_model_single.onnx --saveEngine=voice_model_fp16.engine --fp16 --explicitBatch

# 3) FP32 (si FP16 pose souci)
# trtexec --onnx=voice_model_single.onnx --saveEngine=voice_model_fp32.engine --explicitBatch

# 4) INT8 (optionnel, nécessite calibration)
# trtexec --onnx=voice_model_single.onnx --int8 --calib=<ton_calib_cache> --saveEngine=voice_model_int8.engine --explicitBatch

print("ℹ️ Copie voice_model_single.onnx depuis Drive (exports/) vers la Jetson, puis lance trtexec.")
